In [1]:
# Import dependancies
import ast
import time
import requests
import json
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Read in raw csv
df = pd.read_csv('Data/unprocessed_data.csv')

# Check to make sure it imported
df.head(1)

,id,Title,Movie Link,Year,Duration,MPA,Rating,Votes,budget,grossWorldWide,...,writers,stars,genres,countries_origin,filming_locations,production_companies,Languages,wins,nominations,oscars
0,tt0073195,Jaws,https://www.imdb.com/title/tt0073195,1975,2h 4m,PG,8.1,683K,7000000.0,477220580.0,...,"['Peter Benchley', 'Carl Gottlieb']","['Roy Scheider', 'Robert Shaw', 'Richard Dreyf...","['Monster Horror', 'Sea Adventure', 'Survival'...",['United States'],"[""Water Street, Edgartown, Martha's Vineyard, ...","['Zanuck/Brown Productions', 'Universal Pictur...",['English'],0,20,0


In [3]:
# Check Data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33600 entries, 0 to 33599
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     33600 non-null  object 
 1   Title                  33600 non-null  object 
 2   Movie Link             33600 non-null  object 
 3   Year                   33600 non-null  int64  
 4   Duration               33379 non-null  object 
 5   MPA                    25624 non-null  object 
 6   Rating                 33462 non-null  float64
 7   Votes                  33462 non-null  object 
 8   budget                 11815 non-null  float64
 9   grossWorldWide         18222 non-null  float64
 10  gross_US_Canada        17571 non-null  float64
 11  opening_weekend_Gross  15523 non-null  float64
 12  directors              33241 non-null  object 
 13  writers                32024 non-null  object 
 14  stars                  33127 non-null  object 
 15  ge

In [4]:
# Remove unnecessary columns from df
df_step_1 = df[[
    'id',
    'Title',
    'Year',
    'Rating',
    'Votes',
    'budget',
    'grossWorldWide',
    'gross_US_Canada',
    'opening_weekend_Gross',
    'genres',
    'production_companies',
    'nominations',
    'oscars'
]]

df_step_1.head()

,id,Title,Year,Rating,Votes,budget,grossWorldWide,gross_US_Canada,opening_weekend_Gross,genres,production_companies,nominations,oscars
0,tt0073195,Jaws,1975,8.1,683K,7000000.0,477220580.0,266567580.0,7061513.0,"['Monster Horror', 'Sea Adventure', 'Survival'...","['Zanuck/Brown Productions', 'Universal Pictur...",20,0
1,tt0073629,The Rocky Horror Picture Show,1975,7.4,173K,1200000.0,115798478.0,112892319.0,NaN,"['Dark Comedy', 'Raunchy Comedy', 'Rock Musica...","['Twentieth Century Fox', 'Michael White Produ...",4,0
2,tt0073486,One Flew Over the Cuckoo's Nest,1975,8.7,1.1M,3000000.0,109115366.0,108981275.0,NaN,"['Medical Drama', 'Psychological Drama', 'Drama']","['Fantasy Films', 'N.V. Zvaluw']",15,0
3,tt0072890,Dog Day Afternoon,1975,8.0,279K,1800000.0,50002721.0,50000000.0,NaN,"['Heist', 'True Crime', 'Biography', 'Crime', ...","['Warner Bros.', 'Artists Entertainment Complex']",20,0
4,tt0073692,Shampoo,1975,6.4,15K,4000000.0,49407734.0,49407734.0,NaN,"['Satire', 'Comedy', 'Drama']","['Persky-Bright / Vista', 'Columbia Pictures',...",11,0


In [5]:
# Update names for clarity
df_step_2 = df_step_1.rename(columns= {
    'Title': 'title',
    'Year': 'year',
    'Rating': 'rating',
    'Votes': 'votes',
    'grossWorldWide': 'gross_world_wide',
    'gross_US_Canada': 'gross_us_canada',
    'opening_weekend_Gross': 'opening_weekend_gross'
})

In [6]:
# Filter dataset for all movies after the year 2005
df_step_3 = df_step_2[df_step_2['year'] >= 2005].reset_index()
df_step_3.head()

,index,id,title,year,rating,votes,budget,gross_world_wide,gross_us_canada,opening_weekend_gross,genres,production_companies,nominations,oscars
0,2000,tt1502397,Bad Boys for Life,2020,6.5,187K,90000000.0,426505244.0,206305244.0,62504105.0,"['Buddy Cop', 'Cop Drama', 'Action', 'Comedy',...","['Columbia Pictures', '2.0 Entertainment', 'Do...",15,0
1,2001,tt8332922,A Quiet Place Part II,2020,7.2,296K,61000000.0,297372261.0,160072261.0,47547231.0,"['Alien Invasion', 'Monster Horror', 'Supernat...","['Paramount Pictures', 'Platinum Dunes', 'Sund...",44,1
2,2002,tt3794354,Sonic the Hedgehog,2020,6.5,171K,85000000.0,319715683.0,148974665.0,58018348.0,"['Animal Adventure', 'Buddy Comedy', 'Road Tri...","['Paramount Pictures', 'Sega Sammy Group', 'Or...",12,0
3,2003,tt7713068,Birds of Prey and the Fantabulous Emancipation...,2020,6.1,271K,84500000.0,205537933.0,84172791.0,33010017.0,"['Dark Comedy', 'Superhero', 'Action', 'Comedy...","['Clubhouse Pictures (II)', 'DC Entertainment'...",83,0
4,2004,tt6673612,Dolittle,2020,5.6,73K,175000000.0,251410631.0,77047065.0,21844045.0,"['Animal Adventure', 'Quest', 'Adventure', 'Co...","['Universal Pictures', 'Perfect World Pictures...",9,0


In [7]:
# Drop all rows with missing vlaues
cleaned_df = df_step_3.dropna()
cleaned_df.head()

,index,id,title,year,rating,votes,budget,gross_world_wide,gross_us_canada,opening_weekend_gross,genres,production_companies,nominations,oscars
0,2000,tt1502397,Bad Boys for Life,2020,6.5,187K,90000000.0,426505244.0,206305244.0,62504105.0,"['Buddy Cop', 'Cop Drama', 'Action', 'Comedy',...","['Columbia Pictures', '2.0 Entertainment', 'Do...",15,0
1,2001,tt8332922,A Quiet Place Part II,2020,7.2,296K,61000000.0,297372261.0,160072261.0,47547231.0,"['Alien Invasion', 'Monster Horror', 'Supernat...","['Paramount Pictures', 'Platinum Dunes', 'Sund...",44,1
2,2002,tt3794354,Sonic the Hedgehog,2020,6.5,171K,85000000.0,319715683.0,148974665.0,58018348.0,"['Animal Adventure', 'Buddy Comedy', 'Road Tri...","['Paramount Pictures', 'Sega Sammy Group', 'Or...",12,0
3,2003,tt7713068,Birds of Prey and the Fantabulous Emancipation...,2020,6.1,271K,84500000.0,205537933.0,84172791.0,33010017.0,"['Dark Comedy', 'Superhero', 'Action', 'Comedy...","['Clubhouse Pictures (II)', 'DC Entertainment'...",83,0
4,2004,tt6673612,Dolittle,2020,5.6,73K,175000000.0,251410631.0,77047065.0,21844045.0,"['Animal Adventure', 'Quest', 'Adventure', 'Co...","['Universal Pictures', 'Perfect World Pictures...",9,0


In [8]:
# Check the data
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4378 entries, 0 to 10687
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  4378 non-null   int64  
 1   id                     4378 non-null   object 
 2   title                  4378 non-null   object 
 3   year                   4378 non-null   int64  
 4   rating                 4378 non-null   float64
 5   votes                  4378 non-null   object 
 6   budget                 4378 non-null   float64
 7   gross_world_wide       4378 non-null   float64
 8   gross_us_canada        4378 non-null   float64
 9   opening_weekend_gross  4378 non-null   float64
 10  genres                 4378 non-null   object 
 11  production_companies   4378 non-null   object 
 12  nominations            4378 non-null   int64  
 13  oscars                 4378 non-null   int64  
dtypes: float64(5), int64(4), object(5)
memory usage: 513.0+ KB


## API implamentation
I need to implament the OMDb API, hich will give me some of the same data. 

Here is a list of things I need to do to accompplish this plan:
- I have to check how many requests I am able to make per hour/day to OMDb
- Find relavent information I want to store before making a request
- sort my dataset for only american movies, I am hoping that this will fix some of the currency problems I am facing. 



### Start by filtering the original dataset to only include american movies

In [9]:
# Make a copy of the original DataFrame
df_copy = df.copy()

# Convert the string representations of lists to actual lists
df_copy['countries_origin'] = df_copy['countries_origin'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

# Strip whitespace from items in the 'countries_origin' column
df_copy['countries_origin'] = df_copy['countries_origin'].apply(
    lambda x: [item.strip() for item in x] if isinstance(x, list) else x
)

# Filter for rows where 'United States' is in the 'countries_origin' column
US_movies_df = df_copy[df_copy["countries_origin"].apply(
    lambda x: 'United States' in x if isinstance(x, list) else False
)]

# Display the first 5 rows of the filtered DataFrame
US_movies_df.head(5)


,id,Title,Movie Link,Year,Duration,MPA,Rating,Votes,budget,grossWorldWide,...,writers,stars,genres,countries_origin,filming_locations,production_companies,Languages,wins,nominations,oscars
0,tt0073195,Jaws,https://www.imdb.com/title/tt0073195,1975,2h 4m,PG,8.1,683K,7000000.0,477220580.0,...,"['Peter Benchley', 'Carl Gottlieb']","['Roy Scheider', 'Robert Shaw', 'Richard Dreyf...","['Monster Horror', 'Sea Adventure', 'Survival'...",[United States],"[""Water Street, Edgartown, Martha's Vineyard, ...","['Zanuck/Brown Productions', 'Universal Pictur...",['English'],0,20,0
1,tt0073629,The Rocky Horror Picture Show,https://www.imdb.com/title/tt0073629,1975,1h 40m,R,7.4,173K,1200000.0,115798478.0,...,"[""Richard O'Brien"", 'Jim Sharman']","['Tim Curry', 'Susan Sarandon', 'Barry Bostwick']","['Dark Comedy', 'Raunchy Comedy', 'Rock Musica...","[United Kingdom, United States]","['Oakley Court, Windsor Road, Oakley Green, Wi...","['Twentieth Century Fox', 'Michael White Produ...",['English'],0,4,0
2,tt0073486,One Flew Over the Cuckoo's Nest,https://www.imdb.com/title/tt0073486,1975,2h 13m,R,8.7,1.1M,3000000.0,109115366.0,...,"['Lawrence Hauben', 'Bo Goldman', 'Ken Kesey']","['Jack Nicholson', 'Louise Fletcher', 'Michael...","['Medical Drama', 'Psychological Drama', 'Drama']",[United States],['Oregon State Mental Hospital - 2600 Center S...,"['Fantasy Films', 'N.V. Zvaluw']",['English'],0,15,0
3,tt0072890,Dog Day Afternoon,https://www.imdb.com/title/tt0072890,1975,2h 5m,R,8.0,279K,1800000.0,50002721.0,...,"['Frank Pierson', 'P.F. Kluge', 'Thomas Moore']","['Al Pacino', 'John Cazale', 'Penelope Allen']","['Heist', 'True Crime', 'Biography', 'Crime', ...",[United States],"['285 Prospect Park West, Brooklyn, New York C...","['Warner Bros.', 'Artists Entertainment Complex']",['English'],0,20,0
4,tt0073692,Shampoo,https://www.imdb.com/title/tt0073692,1975,1h 50m,R,6.4,15K,4000000.0,49407734.0,...,"['Robert Towne', 'Warren Beatty']","['Warren Beatty', 'Julie Christie', 'Goldie Ha...","['Satire', 'Comedy', 'Drama']",[United States],"['2270 Bowmont Drive, Beverly Hills, Californi...","['Persky-Bright / Vista', 'Columbia Pictures',...",['English'],0,11,0


In [10]:
print(US_movies_df['countries_origin'].explode().unique())

['United States' 'United Kingdom' 'Japan' 'Czechoslovakia' 'Yugoslavia'
 'Canada' 'Argentina' 'Mexico' 'Philippines' 'Italy' 'Spain' 'Israel'
 'Hungary' 'Sweden' 'Austria' 'Netherlands' 'Belgium' 'Hong Kong'
 'Switzerland' 'France' 'West Germany' 'Taiwan' 'Greenland' 'Australia'
 'Portugal' 'Turkey' 'South Africa' 'China' 'India' 'Ireland' 'Germany'
 'Bahamas' 'Russia' 'Greece' 'New Zealand' 'Norway' 'Poland' 'Indonesia'
 'Iceland' 'Morocco' 'Romania' 'Luxembourg' 'United Arab Emirates'
 'Ukraine' 'Georgia' 'Chile' 'Puerto Rico' 'Thailand' 'Niger' 'Singapore'
 'Brazil' 'Denmark' 'Saudi Arabia' 'Jordan' 'Qatar' 'Finland' 'Lithuania'
 'Rwanda' 'Pakistan' 'Iran' 'Guatemala' 'Vietnam' 'Bulgaria' 'Colombia'
 'Czech Republic' 'Dominican Republic' 'South Korea' 'Cyprus' 'Lebanon'
 'Liberia' 'Occupied Palestinian Territory' 'Kenya' 'Bangladesh' 'Iraq'
 'Peru' 'Cuba' 'Ghana' 'Swaziland' 'Namibia' 'Zimbabwe' 'Cambodia'
 'Serbia' 'Botswana' 'Albania' 'Venezuela' 'Malaysia' 'Sri Lanka'
 'Afghanist

In [11]:
US_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18250 entries, 0 to 33599
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     18250 non-null  object 
 1   Title                  18250 non-null  object 
 2   Movie Link             18250 non-null  object 
 3   Year                   18250 non-null  int64  
 4   Duration               18216 non-null  object 
 5   MPA                    16961 non-null  object 
 6   Rating                 18227 non-null  float64
 7   Votes                  18227 non-null  object 
 8   budget                 9190 non-null   float64
 9   grossWorldWide         12616 non-null  float64
 10  gross_US_Canada        12568 non-null  float64
 11  opening_weekend_Gross  11219 non-null  float64
 12  directors              18228 non-null  object 
 13  writers                17382 non-null  object 
 14  stars                  18135 non-null  object 
 15  genres 

In [12]:
# Create a separate copy of the original DataFrame
movies_df_copy = df.copy()

# Convert the string representation of lists into actual lists
movies_df_copy['countries_origin'] = movies_df_copy['countries_origin'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

# Strip whitespace from the items in the 'countries_origin' column
movies_df_copy['countries_origin'] = movies_df_copy['countries_origin'].apply(
    lambda x: [item.strip() for item in x] if isinstance(x, list) else x
)

# Filter for rows where 'countries_origin' contains only 'United States'
us_only_movies_df = movies_df_copy[movies_df_copy['countries_origin'].apply(
    lambda x: x == ['United States'] if isinstance(x, list) else False
)]

# Display the first 5 rows of the filtered DataFrame
us_only_movies_df.head(5)

,id,Title,Movie Link,Year,Duration,MPA,Rating,Votes,budget,grossWorldWide,...,writers,stars,genres,countries_origin,filming_locations,production_companies,Languages,wins,nominations,oscars
0,tt0073195,Jaws,https://www.imdb.com/title/tt0073195,1975,2h 4m,PG,8.1,683K,7000000.0,477220580.0,...,"['Peter Benchley', 'Carl Gottlieb']","['Roy Scheider', 'Robert Shaw', 'Richard Dreyf...","['Monster Horror', 'Sea Adventure', 'Survival'...",[United States],"[""Water Street, Edgartown, Martha's Vineyard, ...","['Zanuck/Brown Productions', 'Universal Pictur...",['English'],0,20,0
2,tt0073486,One Flew Over the Cuckoo's Nest,https://www.imdb.com/title/tt0073486,1975,2h 13m,R,8.7,1.1M,3000000.0,109115366.0,...,"['Lawrence Hauben', 'Bo Goldman', 'Ken Kesey']","['Jack Nicholson', 'Louise Fletcher', 'Michael...","['Medical Drama', 'Psychological Drama', 'Drama']",[United States],['Oregon State Mental Hospital - 2600 Center S...,"['Fantasy Films', 'N.V. Zvaluw']",['English'],0,15,0
3,tt0072890,Dog Day Afternoon,https://www.imdb.com/title/tt0072890,1975,2h 5m,R,8.0,279K,1800000.0,50002721.0,...,"['Frank Pierson', 'P.F. Kluge', 'Thomas Moore']","['Al Pacino', 'John Cazale', 'Penelope Allen']","['Heist', 'True Crime', 'Biography', 'Crime', ...",[United States],"['285 Prospect Park West, Brooklyn, New York C...","['Warner Bros.', 'Artists Entertainment Complex']",['English'],0,20,0
4,tt0073692,Shampoo,https://www.imdb.com/title/tt0073692,1975,1h 50m,R,6.4,15K,4000000.0,49407734.0,...,"['Robert Towne', 'Warren Beatty']","['Warren Beatty', 'Julie Christie', 'Goldie Ha...","['Satire', 'Comedy', 'Drama']",[United States],"['2270 Bowmont Drive, Beverly Hills, Californi...","['Persky-Bright / Vista', 'Columbia Pictures',...",['English'],0,11,0
6,tt0073026,Funny Lady,https://www.imdb.com/title/tt0073026,1975,2h 16m,PG,6.2,6.1K,NaN,39000000.0,...,"['Jay Presson Allen', 'Arnold Schulman']","['Barbra Streisand', 'James Caan', 'Omar Sharif']","['Biography', 'Comedy', 'Drama', 'Musical', 'R...",[United States],"['Central Station, Oakland, California, USA']","['Columbia Pictures', 'Rastar Pictures', 'Vista']",['English'],0,0,5


In [13]:
us_only_movies_df = us_only_movies_df.dropna()

us_only_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4062 entries, 0 to 33360
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     4062 non-null   object 
 1   Title                  4062 non-null   object 
 2   Movie Link             4062 non-null   object 
 3   Year                   4062 non-null   int64  
 4   Duration               4062 non-null   object 
 5   MPA                    4062 non-null   object 
 6   Rating                 4062 non-null   float64
 7   Votes                  4062 non-null   object 
 8   budget                 4062 non-null   float64
 9   grossWorldWide         4062 non-null   float64
 10  gross_US_Canada        4062 non-null   float64
 11  opening_weekend_Gross  4062 non-null   float64
 12  directors              4062 non-null   object 
 13  writers                4062 non-null   object 
 14  stars                  4062 non-null   object 
 15  genres  

In [14]:
us_movie_ids = us_only_movies_df['id'].tolist()
print(us_movie_ids[:5])

['tt0073195', 'tt0073778', 'tt0076759', 'tt0076666', 'tt0076618']


### I need to take this list and split them up into 5 even parts (I am only allowed 1000 calls a day)

In [21]:
total_ids = len(us_movie_ids)
num_of_sublists = 5
length_of_each = total_ids // num_of_sublists
remainder = total_ids % num_of_sublists

# Create the sublists with unique variable names
us_ids_1 = us_movie_ids[:length_of_each + (1 if remainder > 0 else 0)]
us_ids_2 = us_movie_ids[length_of_each + (1 if remainder > 0 else 0):2 * length_of_each + (1 if remainder > 1 else 0)]
us_ids_3 = us_movie_ids[2 * length_of_each + (1 if remainder > 1 else 0):3 * length_of_each + (1 if remainder > 2 else 0)]
us_ids_4 = us_movie_ids[3 * length_of_each + (1 if remainder > 2 else 0):4 * length_of_each + (1 if remainder > 3 else 0)]
us_ids_5 = us_movie_ids[4 * length_of_each + (1 if remainder > 3 else 0):]

# Print the lengths of each sublist to verify
print(len(us_ids_1), len(us_ids_2), len(us_ids_3), len(us_ids_4), len(us_ids_5))


813 812 811 812 814


In [22]:
print(us_ids_1[:5])
print(us_ids_2[:5])
print(us_ids_3[:5])
print(us_ids_4[:5])
print(us_ids_5[:5])

['tt0073195', 'tt0073778', 'tt0076759', 'tt0076666', 'tt0076618']
['tt0102175', 'tt0101764', 'tt0102517', 'tt0102004', 'tt0102719']
['tt0383060', 'tt0308055', 'tt0469263', 'tt0805526', 'tt0462200']
['tt0821640', 'tt1201167', 'tt1385912', 'tt0976238', 'tt0457400']
['tt0305669', 'tt0286716', 'tt0337741', 'tt0329575', 'tt0257076']


### API Implamentatoin
Now that I have my lists of US-based movies IMDB ids, I want to both create a loop that will pull all relevent information and store it in a different dataframe. I am hoping that this will fix the currency discrepancies. Start by making a call for one movie to get the basic structure of the api-endpoint

### OMDb API JSON Structure Review

In [25]:
# Declare api variables
base_url = "http://www.omdbapi.com/?"
id = us_ids_2[0]
OMDb_api_key = "c6c98d68"

url = f"{base_url}i={id}&apikey={OMDb_api_key}"
print(url)


http://www.omdbapi.com/?i=tt0102175&apikey=c6c98d68


In [26]:
# Perform a GET request for the data
response = requests.get(url)

# Convert the response into JSON format
data = response.json()

# Display the data
pprint(data)

{'Actors': 'Wesley Snipes, Annabella Sciorra, Spike Lee',
 'Awards': '6 wins & 11 nominations total',
 'BoxOffice': '$32,482,682',
 'Country': 'United States',
 'DVD': 'N/A',
 'Director': 'Spike Lee',
 'Genre': 'Drama, Romance',
 'Language': 'English',
 'Metascore': '78',
 'Plot': 'Friends and family of a married Black architect react in different '
         'ways to his affair with an Italian secretary.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMjM5MTQxMDk2Ml5BMl5BanBnXkFtZTgwODQ2MDc3NjE@._V1_SX300.jpg',
 'Production': 'N/A',
 'Rated': 'R',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.6/10'},
             {'Source': 'Rotten Tomatoes', 'Value': '57%'},
             {'Source': 'Metacritic', 'Value': '78/100'}],
 'Released': '07 Jun 1991',
 'Response': 'True',
 'Runtime': '132 min',
 'Title': 'Jungle Fever',
 'Type': 'movie',
 'Website': 'N/A',
 'Writer': 'Spike Lee',
 'Year': '1991',
 'imdbID': 'tt0102175',
 'imdbRating': '6.6',
 'imdbVotes': '20,027'}


### The Movie Database (TMDB) JSON Structure Review

In [27]:
TMDB_api_key = "094cfd0435c22ae4ebc69a98dd79d307"
TMDB_base_url = "https://api.themoviedb.org/3/find/"

In [28]:
# API endpoint for fetching popular movies
url = f"{TMDB_base_url}tt0101787?external_source=imdb_id&api_key={TMDB_api_key}"

# Make the GET request
response = requests.get(url)

# Parse JSON response
data = response.json()

# Pretty print the response
print(json.dumps(data, indent=4))

{
    "movie_results": [
        {
            "backdrop_path": "/1vfyThiuGBuAS8IGs9ovDQMjkLZ.jpg",
            "id": 9079,
            "title": "Dying Young",
            "original_title": "Dying Young",
            "overview": "After she discovers that her boyfriend has betrayed her, Hilary O'Neil is looking for a new start and a new job. She begins to work as a private nurse for a young man suffering from blood cancer. Slowly, they fall in love, but they always know their love cannot last because he is destined to die.",
            "poster_path": "/nLVV8TETjCxWQmIip6t3HO1znVJ.jpg",
            "media_type": "movie",
            "adult": false,
            "original_language": "en",
            "genre_ids": [
                18,
                10749
            ],
            "popularity": 5.1303,
            "release_date": "1991-06-20",
            "video": false,
            "vote_average": 6.681,
            "vote_count": 454
        }
    ],
    "person_results": [],
    "tv_r

### Now, create functions with the parameters being the imdb id array and your api key for the related API
1. Create a function that fetches needed data from the OMDb.<br>
    **Needed Data:**
    1. The Related Website Ratings along with their source
    2. Box Office Earnings
    3. Country of Origin
    4. Director
    5. All Unique Genres
    6. Plot Description
    7. The Letter Rating
    8. Release Date
    9. Total Runtime of movie
    10. The Title
    11. The Year of Release
    12. The IMDB ID
    13. The IMDB Rating (Out of 10)
    14. The amount of IMDB vote recieved
    15. Production Companies

In [29]:
# Function to fetch movie data and append to the DataFrame
def fetch_movie_data(imdb_ids, api_key):
    global api_movie_df  # Ensure we modify the existing DataFrame
    base_url = "http://www.omdbapi.com/"
    
    movie_data = []
    
    for imdb_id in imdb_ids:
        params = {"i": imdb_id, "apikey": api_key}
        response = requests.get(base_url, params=params)
        data = response.json()

        if data.get("Response") == "True":
            # Extract ratings into separate columns
            ratings = {rating["Source"]: rating["Value"] for rating in data.get("Ratings", [])}

            movie_data.append({
                "BoxOffice": data.get("BoxOffice"),
                "Country": data.get("Country"),
                "Director": data.get("Director"),
                "Genre": data.get("Genre").split(", ") if data.get("Genre") else [],
                "Plot": data.get("Plot"),
                "Rated": data.get("Rated"),
                "Released": data.get("Released"),
                "Runtime": data.get("Runtime"),
                "Title": data.get("Title"),
                "Year": data.get("Year"),
                "ImdbID": data.get("imdbID"),
                "imdbRating": data.get("imdbRating"),
                "imdbVotes": data.get("imdbVotes"),
                **ratings  # Unpack the rating dictionary into individual columns
            })
    
    # Convert to DataFrame and append to the existing one
    return pd.DataFrame(movie_data)
   

1. Create a function that fetches needed data from The Move Database API.<br>
    **Needed Data:**
    1. The current IMDB ID
    2. The Supplamentary budget data
    3. Popularity Rating
    4. The Vote Average
    5. The Vote Count 

In [30]:
def fetch_budget_from_tmdb(imdb_ids, api_key):
    base_url = "https://api.themoviedb.org/3/find/"
    movie_data = []

    for imdb_id in imdb_ids:
        try:
            url = f"{base_url}{imdb_id}?external_source=imdb_id&api_key={api_key}"
            response = requests.get(url)
            data = response.json()

            if data.get("movie_results"):
                tmdb_movie = data["movie_results"][0]
                tmdb_id = tmdb_movie["id"]
                
                movie_url = f"https://api.themoviedb.org/3/movie/{tmdb_id}?api_key={api_key}"
                movie_response = requests.get(movie_url)
                movie_details = movie_response.json()

                movie_data.append({
                    "imdb_id": imdb_id,
                    "budget": movie_details.get("budget", 0),
                    "popularity": tmdb_movie.get("popularity", 0),
                    "vote_average": tmdb_movie.get("vote_average", 0),
                    "vote_count": tmdb_movie.get("vote_count", 0)
                })
            else:
                # Ensure missing IMDb IDs are accounted for
                movie_data.append({
                    "imdb_id": imdb_id,
                    "budget": None,
                    "popularity": None,
                    "vote_average": None,
                    "vote_count": None
                })


            time.sleep(0.5)  # Avoid hitting rate limits

        except Exception as e:
            print(f"Error fetching data for {imdb_id}: {e}")

    # Convert to DataFrame and set IMDb ID as index
    return pd.DataFrame(movie_data)

### Make a Looping Request to Both the API's

In [31]:
# The OMDB API Request
OMDb_data = fetch_movie_data(us_ids_2, OMDb_api_key)

In [32]:
TMDB_data = fetch_budget_from_tmdb(us_ids_2, TMDB_api_key)

### Check Both Responses 

In [33]:
OMDb_data.head(3)

,BoxOffice,Country,Director,Genre,Plot,Rated,Released,Runtime,Title,Year,ImdbID,imdbRating,imdbVotes,Internet Movie Database,Rotten Tomatoes,Metacritic
0,"$32,482,682",United States,Spike Lee,"[Drama, Romance]",Friends and family of a married Black architec...,R,07 Jun 1991,132 min,Jungle Fever,1991,tt0102175,6.6,"20,027",6.6/10,57%,78/100
1,"$30,102,717",United States,Sheldon Lettich,"[Action, Crime]",Twin brothers are separated when their parents...,R,09 Aug 1991,110 min,Double Impact,1991,tt0101764,5.6,"45,020",5.6/10,33%,40/100
2,"$26,255,594",United States,Stan Dragoti,"[Comedy, Sport]","Due to NCAA sanctions, the Texas State Univers...",PG-13,27 Sep 1991,108 min,Necessary Roughness,1991,tt0102517,6.2,"13,364",6.2/10,35%,45/100


In [34]:
TMDB_data.head(3)

,imdb_id,budget,popularity,vote_average,vote_count
0,tt0102175,14000000,0.9316,6.4,288
1,tt0101764,15000000,3.1190,6.2,1100
2,tt0102517,0,0.8533,5.5,123


### Combine both dataframes and clean up missing values for csv exporting

In [35]:
# Make a copy of both dataframes
data_1 = OMDb_data.copy()
data_2 = TMDB_data.copy()

# Set the index to be the imdb_id column
data_1.set_index('ImdbID', inplace=True)
data_1.index.name = 'IMDB ID'
data_2.set_index('imdb_id', inplace=True)
data_2.index.name = 'IMDB ID'

# Merge into a single dataframe
combined_data = pd.merge(data_1, data_2, left_index=True, right_index=True, how='inner')

# Get rid of Ratings from OMDb API
normal_data = combined_data.drop(columns=['Internet Movie Database', 'Rotten Tomatoes', 'Metacritic'])

# Dropping columns where there was no budget data found
normal_data = normal_data[normal_data['budget'] != 0]

# Drop extra rows
normal_data = normal_data.dropna(subset=['budget', 'popularity', 'vote_average', 'vote_count'])

# Reset the index
normal_data = normal_data.reset_index()

# Drop na columns
normal_data = normal_data.dropna()

# Check data
normal_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680 entries, 0 to 679
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   IMDB ID       680 non-null    object 
 1   BoxOffice     680 non-null    object 
 2   Country       680 non-null    object 
 3   Director      680 non-null    object 
 4   Genre         680 non-null    object 
 5   Plot          680 non-null    object 
 6   Rated         680 non-null    object 
 7   Released      680 non-null    object 
 8   Runtime       680 non-null    object 
 9   Title         680 non-null    object 
 10  Year          680 non-null    object 
 11  imdbRating    680 non-null    object 
 12  imdbVotes     680 non-null    object 
 13  budget        680 non-null    int64  
 14  popularity    680 non-null    float64
 15  vote_average  680 non-null    float64
 16  vote_count    680 non-null    int64  
dtypes: float64(2), int64(2), object(13)
memory usage: 90.4+ KB


In [37]:
# Export to cvs file
normal_data.to_csv('combined_data_2.csv', index=False)

### Now that we have all of our data, we need to combine it into a single csv file for building our visualizations

In [15]:
movie_df_1 = pd.read_csv('API_data/combined_data_1.csv')
movie_df_2 = pd.read_csv('API_data/combined_data_2.csv')
movie_df_3 = pd.read_csv('API_data/combined_data_3.CSV')
movie_df_4 = pd.read_csv('API_data/combined_data_4.CSV')
movie_df_5 = pd.read_csv('API_data/combined_data_5.CSV')

In [16]:
# Double-check the length of the data
print(len(movie_df_1))
print(len(movie_df_2))
print(len(movie_df_3))
print(len(movie_df_4))
print(len(movie_df_5))

664
680
683
695
674


In [17]:
# Combine the dataframes in order
step_1_combination = pd.concat([movie_df_1, movie_df_2], ignore_index=True)
step_2_combination = pd.concat([movie_df_3, movie_df_4], ignore_index=True)
step_3_combination = pd.concat([step_1_combination, step_2_combination], ignore_index=True)
step_4_combination = pd.concat([step_3_combination, movie_df_5], ignore_index=True)

data = step_4_combination.copy()
print(len(data))


3396


### Now, export the combined data to a new csv file for data visualizations

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3396 entries, 0 to 3395
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   IMDB ID       3396 non-null   object 
 1   BoxOffice     3396 non-null   object 
 2   Country       3396 non-null   object 
 3   Director      3396 non-null   object 
 4   Genre         3396 non-null   object 
 5   Plot          3396 non-null   object 
 6   Rated         3396 non-null   object 
 7   Released      3392 non-null   object 
 8   Runtime       3396 non-null   object 
 9   Title         3396 non-null   object 
 10  Year          3396 non-null   int64  
 11  imdbRating    3394 non-null   float64
 12  imdbVotes     3396 non-null   object 
 13  budget        3396 non-null   float64
 14  popularity    3396 non-null   float64
 15  vote_average  3396 non-null   float64
 16  vote_count    3396 non-null   float64
dtypes: float64(5), int64(1), object(11)
memory usage: 451.2+ KB


In [19]:
data.head()

,IMDB ID,BoxOffice,Country,Director,Genre,Plot,Rated,Released,Runtime,Title,Year,imdbRating,imdbVotes,budget,popularity,vote_average,vote_count
0,tt0073195,"$267,263,625",United States,Steven Spielberg,"['Adventure', 'Drama', 'Horror']",When a massive killer shark unleashes chaos on...,PG,20 Jun 1975,124 min,Jaws,1975,8.1,"689,820",7000000.0,28.925875,7.664,10673.0
1,tt0076759,"$460,998,507",United States,George Lucas,"['Action', 'Adventure', 'Fantasy']",Luke Skywalker joins forces with a Jedi Knight...,PG,25 May 1977,121 min,Star Wars: Episode IV - A New Hope,1977,8.6,"1,498,082",11000000.0,46.792750,8.202,20979.0
2,tt0076666,"$94,213,184",United States,John Badham,"['Drama', 'Music']","Anxious about his future after high school, a ...",R,16 Dec 1977,118 min,Saturday Night Fever,1977,6.8,"90,015",3500000.0,10.705750,6.700,1940.0
3,tt0076618,"$71,215,869",United States,"John Lounsbery, Wolfgang Reitherman, Art Stevens","['Animation', 'Adventure', 'Comedy']",Two mice of the Rescue Aid Society search for ...,G,22 Jun 1977,77 min,The Rescuers,1977,6.9,"71,743",7500000.0,11.815250,6.773,2529.0
4,tt0076009,"$30,749,142",United States,"John Boorman, Rospo Pallenberg",['Horror'],A teenage girl once possessed by a demon finds...,R,17 Jun 1977,117 min,Exorcist II: The Heretic,1977,3.8,"29,325",14000000.0,7.682375,4.500,728.0


### I forgot to pull the production companies data in the new, updated code. Perform a merger on the new data to get the needed information

In [20]:
# Perform the left join
merged_df = data.merge(
    us_only_movies_df[['id', 'production_companies']],  # Selecting only necessary columns
    left_on='IMDB ID',  # Column in 'data'
    right_on='id',  # Column in 'us_only_movies_df'
    how='left'  # Ensuring all rows in 'data' remain
)

# Drop the extra 'id' column from us_only_movies_df (optional)
merged_df = merged_df.drop(columns=['id'])

merged_df.head()

,IMDB ID,BoxOffice,Country,Director,Genre,Plot,Rated,Released,Runtime,Title,Year,imdbRating,imdbVotes,budget,popularity,vote_average,vote_count,production_companies
0,tt0073195,"$267,263,625",United States,Steven Spielberg,"['Adventure', 'Drama', 'Horror']",When a massive killer shark unleashes chaos on...,PG,20 Jun 1975,124 min,Jaws,1975,8.1,"689,820",7000000.0,28.925875,7.664,10673.0,"['Zanuck/Brown Productions', 'Universal Pictur..."
1,tt0076759,"$460,998,507",United States,George Lucas,"['Action', 'Adventure', 'Fantasy']",Luke Skywalker joins forces with a Jedi Knight...,PG,25 May 1977,121 min,Star Wars: Episode IV - A New Hope,1977,8.6,"1,498,082",11000000.0,46.792750,8.202,20979.0,"['Lucasfilm', 'Twentieth Century Fox']"
2,tt0076666,"$94,213,184",United States,John Badham,"['Drama', 'Music']","Anxious about his future after high school, a ...",R,16 Dec 1977,118 min,Saturday Night Fever,1977,6.8,"90,015",3500000.0,10.705750,6.700,1940.0,"['Paramount Pictures', 'Robert Stigwood Organi..."
3,tt0076618,"$71,215,869",United States,"John Lounsbery, Wolfgang Reitherman, Art Stevens","['Animation', 'Adventure', 'Comedy']",Two mice of the Rescue Aid Society search for ...,G,22 Jun 1977,77 min,The Rescuers,1977,6.9,"71,743",7500000.0,11.815250,6.773,2529.0,"['Walt Disney Animation Studios', 'Walt Disney..."
4,tt0076009,"$30,749,142",United States,"John Boorman, Rospo Pallenberg",['Horror'],A teenage girl once possessed by a demon finds...,R,17 Jun 1977,117 min,Exorcist II: The Heretic,1977,3.8,"29,325",14000000.0,7.682375,4.500,728.0,['Warner Bros.']


### Final Cleaning/Processing steps:
    
1. Clean up object columns that should be float64 or int32

In [21]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3396 entries, 0 to 3395
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   IMDB ID               3396 non-null   object 
 1   BoxOffice             3396 non-null   object 
 2   Country               3396 non-null   object 
 3   Director              3396 non-null   object 
 4   Genre                 3396 non-null   object 
 5   Plot                  3396 non-null   object 
 6   Rated                 3396 non-null   object 
 7   Released              3392 non-null   object 
 8   Runtime               3396 non-null   object 
 9   Title                 3396 non-null   object 
 10  Year                  3396 non-null   int64  
 11  imdbRating            3394 non-null   float64
 12  imdbVotes             3396 non-null   object 
 13  budget                3396 non-null   float64
 14  popularity            3396 non-null   float64
 15  vote_average         

In [22]:
merged_df.head(1)

,IMDB ID,BoxOffice,Country,Director,Genre,Plot,Rated,Released,Runtime,Title,Year,imdbRating,imdbVotes,budget,popularity,vote_average,vote_count,production_companies
0,tt0073195,"$267,263,625",United States,Steven Spielberg,"['Adventure', 'Drama', 'Horror']",When a massive killer shark unleashes chaos on...,PG,20 Jun 1975,124 min,Jaws,1975,8.1,"689,820",7000000.0,28.925875,7.664,10673.0,"['Zanuck/Brown Productions', 'Universal Pictur..."


In [23]:
# Convert 'BoxOffice' from string to int64
merged_df['BoxOffice'] = merged_df['BoxOffice'].str.replace('$', '', regex=False).str.replace(',', '', regex=False).astype('int64')

# Convert 'Released' to datetime format
merged_df['Released'] = pd.to_datetime(merged_df['Released'], format='%d %b %Y')

# Convert 'budget' from string to float64
merged_df['budget'] = merged_df['budget'].astype('float64')

# Remove ' min' and convert to integer
merged_df['Runtime'] = merged_df['Runtime'].str.replace(' min', '', regex=False).astype('int64')

# Remove commas from 'imdbVotes' and convert to float64
merged_df['imdbVotes'] = merged_df['imdbVotes'].str.replace(',', '', regex=False).astype('int64')


# Rename columns for clarity
merged_df = merged_df.rename(columns={
    'IMDB ID': 'imdbID',
    'BoxOffice': 'boxOffice',
    'Country': 'country',
    'Director': 'director',
    'Genre': 'genres',
    'Plot': 'plot',
    'Rated': 'rated',
    'Released': 'released',
    'Runtime': 'runtime (min)',
    'Title': 'title',
    'Year': 'year',
    'vote_average': 'voteAverage',
    'vote_count': 'voteCount',
    'production_companies': 'productionCompanies'
})

merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3396 entries, 0 to 3395
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   imdbID               3396 non-null   object        
 1   boxOffice            3396 non-null   int64         
 2   country              3396 non-null   object        
 3   director             3396 non-null   object        
 4   genres               3396 non-null   object        
 5   plot                 3396 non-null   object        
 6   rated                3396 non-null   object        
 7   released             3392 non-null   datetime64[ns]
 8   runtime (min)        3396 non-null   int64         
 9   title                3396 non-null   object        
 10  year                 3396 non-null   int64         
 11  imdbRating           3394 non-null   float64       
 12  imdbVotes            3396 non-null   int64         
 13  budget               3396 non-nul

In [24]:
merged_df.head(1)

,imdbID,boxOffice,country,director,genres,plot,rated,released,runtime (min),title,year,imdbRating,imdbVotes,budget,popularity,voteAverage,voteCount,productionCompanies
0,tt0073195,267263625,United States,Steven Spielberg,"['Adventure', 'Drama', 'Horror']",When a massive killer shark unleashes chaos on...,PG,1975-06-20,124,Jaws,1975,8.1,689820,7000000.0,28.925875,7.664,10673.0,"['Zanuck/Brown Productions', 'Universal Pictur..."


In [26]:
# Now export the Data into the Data folder
merged_df.to_csv('Data/processed_data.csv', index=False)